In [1]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [2]:
import json
from REVIVAL.util import load_json

In [5]:
load_json("/disk2/fli/REVIVAL2/zs/af3/struct_joint/ParLQ/f89a/seed-1_sample-0/summary_confidences.json")

{'chain_iptm': [0.88, 0.88],
 'chain_pair_iptm': [[0.93, 0.88], [0.88, 0.8]],
 'chain_pair_pae_min': [[0.76, 0.85], [1.36, 0.76]],
 'chain_ptm': [0.93, 0.8],
 'fraction_disordered': 0.0,
 'has_clash': 0.0,
 'iptm': 0.88,
 'ptm': 0.92,
 'ranking_score': 0.89}

In [6]:
json_dict = load_json("/disk2/fli/REVIVAL2/zs/af3/struct_joint/ParLQ/f89a/f89a_summary_confidences.json")
json_dict

{'chain_iptm': [0.88, 0.88],
 'chain_pair_iptm': [[0.93, 0.88], [0.88, 0.8]],
 'chain_pair_pae_min': [[0.76, 0.85], [1.36, 0.76]],
 'chain_ptm': [0.93, 0.8],
 'fraction_disordered': 0.0,
 'has_clash': 0.0,
 'iptm': 0.88,
 'ptm': 0.92,
 'ranking_score': 0.89}

In [7]:
json_dict["chain_ptm"]

[0.93, 0.8]

In [12]:
json_dict["chain_pair_iptm"][0][1]

0.88

In [4]:
load_json("/disk2/fli/REVIVAL2/zs/af3/struct_seperate/ParLQ/f89a/f89a_summary_confidences.json")

{'chain_iptm': [0.84, 0.66, 0.7],
 'chain_pair_iptm': [[0.94, 0.8, 0.88], [0.8, 0.04, 0.52], [0.88, 0.52, 0.82]],
 'chain_pair_pae_min': [[0.76, 2.39, 0.84],
  [5.38, 0.76, 5.9],
  [1.72, 3.14, 0.76]],
 'chain_ptm': [0.94, 0.04, 0.82],
 'fraction_disordered': 0.0,
 'has_clash': 0.0,
 'iptm': 0.88,
 'ptm': 0.93,
 'ranking_score': 0.89}

In [16]:
import pandas as pd
from scipy.stats import spearmanr

In [22]:
import pandas as pd
import numpy as np
from glob import glob
from scipy.stats import spearmanr
from REVIVAL.util import get_file_name

# Initialize an empty list to store results
results = []

# Loop through the CSV files and calculate Spearman correlation
for lib in sorted(glob("/disk2/fli/REVIVAL2/data/meta/not_scaled/*.csv")):
    lib_name = get_file_name(lib)
    lib_df = pd.read_csv(lib)
    af3_df = pd.read_csv(f'/disk2/fli/REVIVAL2/zs/af3/score_joint/{lib_name}.csv')
    df_nan = af3_df[af3_df.isna().any(axis=1)]
    if len(df_nan) > 0:
        print(f"NaN values found in {lib_name} for af3_df")
        print(len(df_nan))
    avg_c = [c for c in af3_df.columns if "avg" in c and "disordered" not in c]
    std_c = [c for c in af3_df.columns if "std" in c and "disordered" not in c]
    agg_c = [c for c in af3_df.columns if "agg" in c and "disordered" not in c]

    # Merge dataframes on the 'var' column
    merged_df = pd.merge(af3_df[["var"] + avg_c + std_c], lib_df[["var", "fitness"]], on="var", how="outer")

    # Calculate Spearman correlation for each column in avg_c
    for c in avg_c:
        lib_df_valid = merged_df.copy()
        valid_rows = lib_df_valid[["fitness", c]].dropna()
        correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
        # correlation, p_value = spearmanr(merged_df["fitness"].values, merged_df[c].values)
        if "agg" not in c:
            std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })
        else:
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                "variability": np.nan,
                "Spearman_Correlation ": correlation,
                "P_Value": p_value
        })


# Convert results into a DataFrame and display it as a table
results_df = pd.DataFrame(results)
results_df

/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficien

NaN values found in Rma-CB for af3_df
18
NaN values found in Rma-CSi for af3_df
18


/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/341136321.py:30: ConstantInputWarning: An input array is constant; the correlation coefficien

,Library,Score_Type,variability,Spearman_Correlation,P_Value
0,ParLQ,ranking_score_avg,0.016154,0.345636,3.395007e-15
1,ParLQ,ptm_avg,0.005519,0.257161,7.677797e-09
2,ParLQ,iptm_avg,0.018891,0.346173,3.058197e-15
3,ParLQ,chain_ptm_A_avg,0.001960,0.143946,1.398825e-03
4,ParLQ,chain_iptm_A_avg,0.018891,0.346173,3.058197e-15
...,...,...,...,...,...
177,Rma-CSi,chain_ptm_B_avg,NaN,0.312909,2.591564e-04
178,Rma-CSi,chain_iptm_B_avg,NaN,0.288446,7.963713e-04
179,Rma-CSi,chain_pae_min_BA_avg,NaN,-0.298585,5.060569e-04
180,Rma-CSi,chain_iptm_BA_avg,NaN,0.288446,7.963713e-04


In [23]:
results_df.Score_Type.unique()

array(['ranking_score_avg', 'ptm_avg', 'iptm_avg', 'chain_ptm_A_avg',
       'chain_iptm_A_avg', 'chain_pae_min_AA_avg', 'chain_pae_min_AB_avg',
       'chain_iptm_AB_avg', 'chain_ptm_B_avg', 'chain_iptm_B_avg',
       'chain_pae_min_BA_avg', 'chain_iptm_BA_avg',
       'chain_pae_min_BB_avg'], dtype=object)

In [19]:
results_df.loc[results_df.groupby("Library")["Spearman_Correlation"].idxmax()]

,Library,Score_Type,variability,Spearman_Correlation,P_Value
2,ParLQ,iptm_avg,0.018891,0.346173,3.058197e-15
23,PfTrpB-4bromo,chain_ptm_B_avg,0.010843,0.284960,6.979952e-06
29,PfTrpB-4cyano,ptm_avg,0.001904,0.297909,2.500511e-06
51,PfTrpB-56chloro,chain_ptm_B_avg,0.010101,0.227565,3.691277e-04
57,PfTrpB-5bromo,ptm_avg,0.002024,0.231984,2.808926e-04
71,PfTrpB-5chloro,ptm_avg,0.002134,0.193617,2.538352e-03
85,PfTrpB-5cyano,ptm_avg,0.001889,0.083419,1.968655e-01
99,PfTrpB-5iodo,ptm_avg,0.001836,0.210031,1.037067e-03
121,PfTrpB-6chloro,chain_ptm_B_avg,0.009180,0.313695,6.675787e-07
127,PfTrpB-7bromo,ptm_avg,0.002015,0.260780,3.172204e-02


In [25]:
import pandas as pd
import numpy as np
from glob import glob
from scipy.stats import spearmanr
from REVIVAL.util import get_file_name

# Initialize an empty list to store results
seperate_results = []

# Loop through the CSV files and calculate Spearman correlation
for lib in [
    "/disk2/fli/REVIVAL2/data/meta/not_scaled/ParLQ.csv",
    "/disk2/fli/REVIVAL2/data/meta/not_scaled/Rma-CB.csv",
    "/disk2/fli/REVIVAL2/data/meta/not_scaled/Rma-CSi.csv"
    ]:
    lib_name = get_file_name(lib)
    lib_df = pd.read_csv(lib)
    af3_df = pd.read_csv(f'/disk2/fli/REVIVAL2/zs/af3/score_seperate/{lib_name}.csv')
    df_nan = af3_df[af3_df.isna().any(axis=1)]
    if len(df_nan) > 0:
        print(f"NaN values found in {lib_name} for af3_df")
        print(len(df_nan))
    avg_c = [c for c in af3_df.columns if "avg" in c and "disordered" not in c]
    std_c = [c for c in af3_df.columns if "std" in c and "disordered" not in c]
    agg_c = [c for c in af3_df.columns if "agg" in c and "disordered" not in c]

    # Merge dataframes on the 'var' column
    merged_df = pd.merge(af3_df[["var"] + avg_c + std_c], lib_df[["var", "fitness"]], on="var", how="outer")

    # Calculate Spearman correlation for each column in avg_c
    for c in avg_c:
        lib_df_valid = merged_df.copy()
        valid_rows = lib_df_valid[["fitness", c]].dropna()
        correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
        # correlation, p_value = spearmanr(merged_df["fitness"].values, merged_df[c].values)
        if "agg" not in c:
            std_col = merged_df[c.replace("avg", "std")].values
            seperate_results.append({
                "Library": lib_name,
                "Score_Type": c,
                "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })
        else:
            seperate_results.append({
                "Library": lib_name,
                "Score_Type": c,
                "variability": np.nan,
                "Spearman_Correlation ": correlation,
                "P_Value": p_value
        })


# Convert results into a DataFrame and display it as a table
seperate_results_df = pd.DataFrame(seperate_results)
seperate_results_df

NaN values found in Rma-CB for af3_df
16
NaN values found in Rma-CSi for af3_df
15


/tmp/ipykernel_3142185/1058937875.py:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/1058937875.py:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/1058937875.py:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/1058937875.py:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
/tmp/ipykernel_3142185/1058937875.py:34: ConstantInputWarning: An input array is constant; the correlation coeff

,Library,Score_Type,variability,Spearman_Correlation,P_Value
0,ParLQ,ranking_score_avg,0.005889,0.338275,1.393332e-14
1,ParLQ,ptm_avg,0.002801,0.262727,3.528431e-09
2,ParLQ,iptm_avg,0.006809,0.338146,1.427638e-14
3,ParLQ,chain_ptm_A_avg,0.001830,0.162490,3.041771e-04
4,ParLQ,chain_iptm_A_avg,0.011352,0.188348,2.714092e-05
...,...,...,...,...,...
67,Rma-CSi,chain_pae_min_CA_avg,NaN,-0.440608,8.894231e-08
68,Rma-CSi,chain_iptm_CA_avg,NaN,0.445878,5.975626e-08
69,Rma-CSi,chain_pae_min_CB_avg,NaN,-0.127260,1.413288e-01
70,Rma-CSi,chain_iptm_CB_avg,NaN,0.172584,4.532314e-02


In [26]:
seperate_results_df.loc[seperate_results_df.groupby("Library")["Spearman_Correlation"].idxmax()]

,Library,Score_Type,variability,Spearman_Correlation,P_Value
9,ParLQ,chain_iptm_AC_avg,0.004996,0.478716,1.961178e-29
33,Rma-CB,chain_iptm_AC_avg,NaN,0.435330,1.466828e-07
57,Rma-CSi,chain_iptm_AC_avg,NaN,0.445878,5.975626e-08
